In [1]:
import datetime
import sqlalchemy
import random
import pandas as pd
import numpy as np
import os
import time
import pandas as pd
import xlwt
from xlwt.Workbook import *
from pandas import ExcelWriter
import xlsxwriter

#IMPORT PARA EL MANEJO DE SQL ALCHEMY
from datetime import datetime
import datetime
from sqlalchemy import join
from sqlalchemy.sql import select
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy.sql import exists
from sqlalchemy import Table, Column, Integer, DateTime, Boolean, Float, String, MetaData, ForeignKey, Date
from sqlalchemy.sql import text
from sqlalchemy.sql import and_, or_, not_
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship   

import openpyxl
from openpyxl import load_workbook
import datetime
from pandas import DataFrame

In [2]:
#leer archivos

archivo = pd.read_excel('ClientesAfectados.xlsx',index_col=None)
archivo_periodos = pd.read_excel('ClientesAfectados.xlsx',sheet_name='lectura_omar',index_col=None)
archivo2 = pd.read_excel('base_madre_enel_v2.xlsx', sheet_name='Origen Cliente')
madre_enel_v2 = pd.read_excel('base_madre_enel_v2.xlsx', sheet_name='Resumen Dadores')
archivo_fotos = pd.read_excel('resultado_analisis_fotos_v3.xlsx',)

df_clientes = pd.DataFrame(archivo)
df_medidores = pd.DataFrame(archivo2)
df_enel = pd.DataFrame(madre_enel_v2)
df_clave_lectura = pd.DataFrame(archivo_periodos)
df_lectura_foto = pd.DataFrame(archivo_fotos)

In [3]:
#Conexión sql server
SERVER = 'DESKTOP-OD7PHAU'
DATABASE = 'CLIENTES_V2'
DRIVER = 'SQL Server Native Client 11.0'
USERNAME = 'ksandoval'
PASSWORD = 'pegasus2020'
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

In [4]:
engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

In [5]:
meta = MetaData()
#variables de Session
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
#creacion de las clases - tablas
class Receptor(Base):
    __tablename__ = "Receptores"

    id_receptor = Column("id_receptor", Integer, primary_key=True)
    nombre_receptor = Column("nombre_receptor", String(255))
    
    def __repr__(self):
        return "Receptor(id_receptor={}, nombre_receptor={})".format(self.id_receptor, self.nombre_receptor)
    
    
    
    
class Dador(Base):
    __tablename__ = "Dadores"
    
    id_dador = Column("id_dador", Integer, primary_key=True)
    receptor_id = Column('receptor_id', Integer, ForeignKey('Receptores.id_receptor'))
    
    def __repr__(self):
        return "Receptor(id_dador={}, receptor_id={})".format(self.id_dador, self.receptor_id)
    
    
    
    
class Medidor(Base):
    __tablename__ = "Medidores"

    id_medidor = Column("id_medidor", Integer, primary_key=True)
    numero_medidor = Column("numero_medidor", Integer)
    tipo_medidor = Column("tipo_medidor", String(255))
    estado_medidor = Column("estado_medidor", String(255))
    capacidad_empalme = Column("capacidad_empalme", String(255))
    consumo_estimado_invierno = Column("consumo_estimado_invierno",  String(255))
    consumo_estimado_verano = Column("consumo_estimado_verano",  String(255))
    propiedad_medidor = Column("propiedad_medidor", String(255))
    tarifa = Column("tarifa", String(255))
    dador_id = Column('dador_id', Integer, ForeignKey('Dadores.id_dador'))
        
    def __repr__(self):
        return "Medidor(id_medidor={}, numero_medidor={},tipo_medidor={},estado_medidor={},capacidad_empalme={},consumo_estimado_invierno={},consumo_estimado_verano={},propiedad_medidor={},tarifa={},dador_id={})".format(
                        self.id_medidor, self.numero_medidor, self.tipo_medidor, self.estado_medidor,self.capacidad_empalme,
                        self.consumo_estimado_invierno,self.consumo_estimado_verano, self.propiedad_medidor,self.tarifa, self.dador_id) 
    
    
    
class Periodo(Base):
    __tablename__ = "Periodos"

    id_periodo = Column("id_periodo", Integer, primary_key=True, autoincrement=True)
    dia = Column("dia", Integer)
    mes = Column("mes", Integer)
    anio = Column("anio", Integer)
    fecha_periodo = Column("fecha_periodo",  Date)
    estado_lectura = Column("estado_lectura", String(255))
    foto_problema_lectura = Column("foto_problema_lectura", String(255))
    dador_id = Column('dador_id', Integer, ForeignKey('Dadores.id_dador'))
    
    def __repr__(self):
        return "Periodo(id_periodo={}, dia={}, mes={}, anio={}, fecha_periodo={},estado_lectura={},foto_problema_lectura={},dador_id={})".format(
                self.id_periodo, self.dia, self.mes, self.anio, self.fecha_periodo, self.estado_lectura,self.foto_problema_lectura, self.dador_id)
    

    
    
class Factura(Base):
    __tablename__ = "Facturas"

    id_factura = Column("id_factura", Integer, primary_key=True)
    suministro_electrico = Column("suministro_electrico", String(500))
    punto_suministro = Column("punto_suministro", String(500))
    interes_adicional = Column("interes_adicional", Float)
    energia_adicional = Column("energia_adicional", Float)
    nota_credito_emitida = Column("nota_credito_emitida",Boolean, default=False)
    dador_id = Column('dador_id', Integer, ForeignKey('Dadores.id_dador'))
        
    def __repr__(self):
        return "Factura(id_factura={}, suministro_electrico={}, punto_suministro={}, interes_adicional={}, energia_adicional={}, nota_credito_emitida={}, dador_id={})".format(
                self.id_factura, self.suministro_electrico, self.punto_suministro, self.interes_adicional, self.energia_adicional, self.nota_credito_emitida, self.dador_id)  
    

In [7]:
#se crean las tablan en memoria
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [8]:
# Creamos el objeto conexión, reset de variables
con = engine.connect()

In [9]:
#Receptores
receptor1 = Receptor(id_receptor =1594986, nombre_receptor =' AGUAS ANDINAS S.A.')
receptor2 = Receptor(id_receptor =1595042, nombre_receptor =' AGUAS ANDINAS S.A.')
receptor3 = Receptor(id_receptor =1677038, nombre_receptor =' AGUAS CORDILLERA S.A.')
receptor4 = Receptor(id_receptor =1677039, nombre_receptor =' AGUAS MANQUEHUE S.A.')
receptor5 = Receptor(id_receptor =2878763, nombre_receptor ='AGUAS MANQUEHUE S.A.')

#añadir datos a Session de insert
session.add(receptor1)
session.add(receptor2)
session.add(receptor3)
session.add(receptor4)
session.add(receptor5)


#guardar
session.commit()
data_r = session.query(Receptor).all()
pprint(data_r)

[Receptor(id_receptor=1594986, nombre_receptor= AGUAS ANDINAS S.A.),
 Receptor(id_receptor=1595042, nombre_receptor= AGUAS ANDINAS S.A.),
 Receptor(id_receptor=1677038, nombre_receptor= AGUAS CORDILLERA S.A.),
 Receptor(id_receptor=1677039, nombre_receptor= AGUAS MANQUEHUE S.A.),
 Receptor(id_receptor=2878763, nombre_receptor=AGUAS MANQUEHUE S.A.)]


In [10]:
for indice_fila, fila in df_clientes.iterrows():
    total_dadores = []
          
    existe_indice_fila = pd.read_sql('SELECT COUNT (*) FROM Dadores where id_dador =' + str(fila['DADOR']), engine)
    valor_indice_fila = str(existe_indice_fila.loc[0][''])
    if int(valor_indice_fila) < 1:
        dador_indice_fila = Dador(id_dador = fila['DADOR'], receptor_id = fila['RECEPTOR'])   
        total_dadores.append(dador_indice_fila)
    
    
    #añadir datos a Session de insert
    #session.add(dador_indice_fila)
    session.add_all(total_dadores)
    session.commit()
    data_d = session.query(Dador).all()

In [11]:
#Medidor
for indice_fila, fila in df_medidores.iterrows():
    total_medidores = []
    
    existe_indice_fila = pd.read_sql('SELECT COUNT (*) FROM Dadores where id_dador =' + str(fila['Dador']), engine)
    valor_indice_fila = str(existe_indice_fila.loc[0][''])
    
    if int(valor_indice_fila) > 0:
        
        if str(fila['Nro medidor']).isnumeric() == True:
            if str(fila['Cant. Meses Verano']) != '#N/D':
                
                medidor_indice_fila = Medidor(  id_medidor = fila['Nro medidor'],
                                                numero_medidor = fila['Nro medidor'],
                                                tipo_medidor = fila['Fase'],
                                                estado_medidor = fila['Estado de medidor'],
                                                capacidad_empalme = fila['Capacidad Empalme'],
                                                consumo_estimado_invierno = 'Total Meses Invierno: ' + str(fila['Cant. Meses Invierno']) + ' Total Consumo Estimado Invierno: ' + str(fila['SUMA CONSUMOS TOTALES INVIERNO']), #agregar consumo estimnado
                                                consumo_estimado_verano = 'Total Meses Verano: ' + str(fila['Cant. Meses Verano']) + ' Total Consumo Estimado Verano: ' + str(fila['SUMA CONSUMOS TOTALES VERANO']), #agregar consumo estimnado
                                                propiedad_medidor = str(fila['Propiedad de medidor']),
                                                tarifa = fila['Tarifa'],
                                                dador_id = fila['Dador'])

                total_medidores.append(medidor_indice_fila)
            else:
                medidor_indice_fila = Medidor(  id_medidor = fila['Nro medidor'],
                                                numero_medidor = fila['Nro medidor'],
                                                tipo_medidor = fila['Fase'],
                                                estado_medidor = fila['Estado de medidor'],
                                                capacidad_empalme = fila['Capacidad Empalme'],
                                                consumo_estimado_invierno = '#N/D', #agregar consumo estimnado
                                                consumo_estimado_verano = '#N/D', #agregar consumo estimnado
                                                propiedad_medidor = str(fila['Propiedad de medidor']),
                                                tarifa = fila['Tarifa'],
                                                dador_id = fila['Dador'])
                total_medidores.append(medidor_indice_fila)

        else:
            medidor_indice_fila = Medidor(  id_medidor = int(indice_fila*10+1),
                                            numero_medidor = 0.0,
                                            tipo_medidor = fila['Fase'],
                                            estado_medidor = fila['Estado de medidor'],
                                            capacidad_empalme = fila['Capacidad Empalme'],
                                            consumo_estimado_invierno = '#N/D', #agregar consumo estimnado
                                            consumo_estimado_verano = '#N/D', #agregar consumo estimnado
                                            propiedad_medidor = str(fila['Propiedad de medidor']),
                                            tarifa = fila['Tarifa'],
                                            dador_id = fila['Dador'])
            
            total_medidores.append(medidor_indice_fila)
    


    #añadir datos a Session de insert
    #session.add(contrato_indice_fila)
    session.add_all(total_medidores)
    session.commit()
    data_c = session.query(Medidor).all()

In [12]:
total_fechas_periodos = []
for a in range(2016,2019):
    for m in range(1,13):
        mes = str(m)
        if m < 10:
            mes_actual = str("0" + mes)
        else:
            mes_actual = str(mes)
                            
        anio = str(a)
        fecha_periodo = str('01-' + mes_actual + '-' + anio)
        #print(fecha_periodo)
        total_fechas_periodos.append(fecha_periodo)
        
print(total_fechas_periodos[0])
f = str(total_fechas_periodos[0])
fecha_cut = f[6:10]     
fecha_cut

01-01-2016


'2016'

In [13]:
archivo_per = pd.read_excel('ClientesAfectados.xlsx', header=None,usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37])
df_fechas = pd.DataFrame(archivo_per)

In [14]:
#Periodos            
for indice_fila, fila in df_lectura_foto.iterrows():
    total_periodos = []
    
    existe_indice_fila = pd.read_sql('SELECT COUNT (*) FROM Dadores where id_dador =' + str(fila['DADOR']), engine)
    valor_indice_fila = str(existe_indice_fila.loc[0][''])
    
    if int(valor_indice_fila) > 0:
        
         if int(str(fila['Fotografía 1'])) > 0:
                
                for contador in range(0,35):

                    #fecha = str(fila)
                    #fecha_cut = fecha[0:10]    
                    f = str(total_fechas_periodos[contador])
                    anio = f[6:10]     
                    periodo_indice_fila = Periodo( dia=1, mes=0, anio=str(anio),
                                                    fecha_periodo= str(total_fechas_periodos[contador]),
                                                    estado_lectura= str(fila[contador+2]),
                                                    foto_problema_lectura = str(fila['Fotografía 1']),
                                                    dador_id = fila['DADOR'])
                    total_periodos.append(periodo_indice_fila)
                
         else:
             for contador in range(0,35):

                    #fecha = str(fila)
                    #fecha_cut = fecha[0:10]    
                    f = str(total_fechas_periodos[contador])
                    anio = f[6:10]     
                    periodo_indice_fila = Periodo( dia=1, mes=0, anio=str(anio),
                                                    fecha_periodo= str(total_fechas_periodos[contador]),
                                                    estado_lectura= str(fila[contador+2]),
                                                    foto_problema_lectura = 'SIN FOTO',
                                                    dador_id = fila['DADOR'])
                    total_periodos.append(periodo_indice_fila)
            

    #session.add(periodo_indice_fila)
    #session.bulk_save_objects(total_periodos)
    session.add_all(total_periodos)
    session.commit()
    data_p = session.query(Periodo).all()

In [ ]:
'''Periodos            
for indice_fila, fila in df_clientes.iterrows():
    total_periodos = []
    
    existe_indice_fila = pd.read_sql('SELECT COUNT (*) FROM Dadores where id_dador =' + str(fila['DADOR']), engine)
    valor_indice_fila = str(existe_indice_fila.loc[0][''])
    
    if int(valor_indice_fila) > 0:
        
            for contador in range(0,36):
                
                #fecha = str(fila)
                #fecha_cut = fecha[0:10]    
                f = str(total_fechas_periodos[contador])
                anio = f[6:10]     
                periodo_indice_fila = Periodo( dia=1, mes=0, anio=str(anio),
                                                fecha_periodo= str(total_fechas_periodos[contador]),
                                                estado_lectura= str(fila[contador+2]), 
                                                dador_id = fila['DADOR'])
                total_periodos.append(periodo_indice_fila)

    #session.add(periodo_indice_fila)
    #session.bulk_save_objects(total_periodos)
    session.add_all(total_periodos)
    session.commit()
    data_p = session.query(Periodo).all()'''

In [ ]:
#Facturas
total_facturas = []
for indice_fila, fila in df_enel.iterrows():
    #total_facturas = []
    existe_indice_fila = pd.read_sql('SELECT COUNT (*) FROM Dadores where id_dador =' + str(fila['DADOR']), engine)
    valor_indice_fila = str(existe_indice_fila.loc[0][''])
    
    if int(valor_indice_fila) > 0:
        
        fecha = str(fila['MES FACTURACION'])
        fecha_cut = fecha[0:10]
        #fecha_formateada = datetime.datetime.strptime(fecha_cut, "%Y-%m-%d")
        factura_indice_fila = Factura(id_factura = indice_fila,
                                      suministro_electrico =  str(fecha_cut) + ' CONSUMO TOTAL ENERGIA: ' + str(fila['CONSUMO TOTAL']),
                                      punto_suministro = 'SI',
                                      interes_adicional = False,
                                      energia_adicional = 0,
                                      nota_credito_emitida = False,
                                      dador_id = fila['DADOR'])   
        
    
    total_facturas.append(factura_indice_fila)    
    #añadir datos a Session de insert
    #session.add(factura_indice_fila)
    session.add_all(total_facturas)
    session.commit()
    data_f = session.query(Factura).all()

In [ ]:
pprint(session.dirty)
session.rollback()
pprint(session.dirty)

In [ ]:
#Se importa OPENPYXL
!pip install openpyxl

In [ ]:
 #Armando la Query con STRING, apuntar a la Tabla 'PLURAL'
df_dador_receptor = pd.DataFrame(columns= ['id_dador  ', 'receptor_id']) 
for i, fila in df_enel.iterrows():
        dadores_i = pd.read_sql('SELECT * FROM Dadores WHERE Dadores.receptor_id =' + str(fila['RECEPTOR']), engine)
        df_dador_receptor = df_dador_receptor.append(dadores_i, sort=False)

In [ ]:
#Resultado de las QUERYS validados contra plantillas de información de ENEL
q_dadores_r1 = pd.read_sql('SELECT DISTINCT * FROM Dadores LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor WHERE Dadores.receptor_id=1594986 ORDER BY Dadores.id_dador', engine)
q_dadores_r2 = pd.read_sql('SELECT DISTINCT * FROM Dadores LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor  WHERE Dadores.receptor_id=1595042 ORDER BY Dadores.id_dador', engine)
q_dadores_r3 = pd.read_sql('SELECT DISTINCT * FROM Dadores LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor WHERE Dadores.receptor_id=1677038 ORDER BY Dadores.id_dador', engine)
q_dadores_r4 = pd.read_sql('SELECT DISTINCT * FROM Dadores LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor WHERE Dadores.receptor_id=1677039 ORDER BY Dadores.id_dador', engine)
q_dadores_r5 = pd.read_sql('SELECT DISTINCT * FROM Dadores LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor WHERE Dadores.receptor_id=2878763 ORDER BY Dadores.id_dador', engine)

In [ ]:
total_1 = pd.DataFrame(columns= ['Item', 'Resultado']) 

for i, fila in q_dadores_r1.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)

In [ ]:
total_1 = pd.DataFrame(columns= ['Item', 'Resultado']) 

for i, fila in q_dadores_r1.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)
    #q_contratos_i = pd.read_sql('SELECT Contratos.contrato_firmado, Contratos.numero_medidor, Contratos.tipo_medidor, Contratos.capacidad_empalme,Contratos.consumo_estimado_invierno,Contratos.consumo_estimado_verano,Contratos.propiedad_medidor,Contratos.tarifa,Contratos.observacion, Contratos.dador_id FROM Contratos WHERE Contratos.dador_id='+str(fila['id_dador']), engine)
    q_contratos_i = pd.read_sql('SELECT Medidores.id_medidor, Medidores.numero_medidor, Medidores.tipo_medidor, Medidores.estado_medidor, Medidores.capacidad_empalme,Medidores.consumo_estimado_invierno,Medidores.consumo_estimado_verano,Medidores.propiedad_medidor,Medidores.tarifa, Medidores.dador_id FROM Medidores WHERE Medidores.dador_id='+str(fila['id_dador']), engine)
    q_facturas_i = pd.read_sql('SELECT * FROM Facturas WHERE Facturas.dador_id=' + str(fila['id_dador']) , engine)
    q_periodos_i_2016 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2016', engine)
    q_periodos_i_2017 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2017', engine)
    q_periodos_i_2018 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2018', engine)
  
    Cliente_i = {
                    'Item':['ID_Cliente_Dador',
                            'Posee_Contrato_Firmado',
                            'Observación',
                            'N_Medidor',
                            'Tipo_Medidor',
                            'Capacidad_Empalme',
                            'Consumo_Estimado_Invierno',
                            'Consumo_Estimado_Verano',
                            'Propiedad_Medidor',
                            'Tarifa_Definida',
                            'Periodo_SLectura_SFacturacion_2016',
                            'Periodo_SLectura_SFacturacion_2017',
                            'Periodo_SLectura_SFacturacion_2018',
                            'Suministro_Electrico',
                            'Punto_Suministro',
                            'Interes_Energia_Adicional',
                            'Nota_Credito_Emitida',
                             ''],
                'Resultado': [str(fila['id_dador']),
                              #q_contratos_i['contrato_firmado'][0],
                              'NO',
                              str('MEDIDOR ' + str(q_contratos_i['estado_medidor'][0])),
                              str(q_contratos_i['id_medidor'][0]),
                              q_contratos_i['tipo_medidor'][0],
                              q_contratos_i['capacidad_empalme'][0],
                              q_contratos_i['consumo_estimado_invierno'][0],
                              q_contratos_i['consumo_estimado_verano'][0],
                              str(fila['id_dador']) + ' - ' + str(fila['nombre_receptor']) + str(q_contratos_i['propiedad_medidor'][0]),
                              q_contratos_i['tarifa'][0],
                              q_periodos_i_2016,
                              q_periodos_i_2017,
                              q_periodos_i_2018,
                              q_facturas_i['suministro_electrico'],
                              q_facturas_i['punto_suministro'],
                              q_facturas_i['energia_adicional'],
                              q_facturas_i['nota_credito_emitida'],
                              '' ]
              }
    
    df_r1_i = pd.DataFrame(Cliente_i)
    total_1 = total_1.append(df_r1_i)
    
    
    
total_2 = pd.DataFrame(columns= ['Item', 'Resultado']) 
for i, fila in q_dadores_r2.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)
    #q_contratos_i = pd.read_sql('SELECT Contratos.contrato_firmado, Contratos.numero_medidor, Contratos.tipo_medidor, Contratos.capacidad_empalme,Contratos.consumo_estimado_invierno,Contratos.consumo_estimado_verano,Contratos.propiedad_medidor,Contratos.tarifa,Contratos.observacion, Contratos.dador_id FROM Contratos WHERE Contratos.dador_id='+str(fila['id_dador']), engine)
    q_contratos_i = pd.read_sql('SELECT Medidores.id_medidor, Medidores.numero_medidor, Medidores.tipo_medidor, Medidores.estado_medidor, Medidores.capacidad_empalme,Medidores.consumo_estimado_invierno,Medidores.consumo_estimado_verano,Medidores.propiedad_medidor,Medidores.tarifa, Medidores.dador_id FROM Medidores WHERE Medidores.dador_id='+str(fila['id_dador']), engine)
    q_facturas_i = pd.read_sql('SELECT * FROM Facturas WHERE Facturas.dador_id=' + str(fila['id_dador']) , engine)
    q_periodos_i_2016 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2016', engine)
    q_periodos_i_2017 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2017', engine)
    q_periodos_i_2018 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2018', engine)
  
    Cliente_i = {
                    'Item':['ID_Cliente_Dador',
                            'Posee_Contrato_Firmado',
                            'Observación',
                            'N_Medidor',
                            'Tipo_Medidor',
                            'Capacidad_Empalme',
                            'Consumo_Estimado_Invierno',
                            'Consumo_Estimado_Verano',
                            'Propiedad_Medidor',
                            'Tarifa_Definida',
                            'Periodo_SLectura_SFacturacion_2016',
                            'Periodo_SLectura_SFacturacion_2017',
                            'Periodo_SLectura_SFacturacion_2018',
                            'Suministro_Electrico',
                            'Punto_Suministro',
                            'Interes_Energia_Adicional',
                            'Nota_Credito_Emitida',
                             ''],
                'Resultado': [str(fila['id_dador']),
                               #q_contratos_i['contrato_firmado'][0],
                              'NO',
                               str('MEDIDOR ' + str(q_contratos_i['estado_medidor'][0])),
                              str(q_contratos_i['id_medidor'][0]),
                              q_contratos_i['tipo_medidor'][0],
                              q_contratos_i['capacidad_empalme'][0],
                              q_contratos_i['consumo_estimado_invierno'][0],
                              q_contratos_i['consumo_estimado_verano'][0],
                              str(fila['id_dador']) + ' - ' + str(fila['nombre_receptor']) + str(q_contratos_i['propiedad_medidor'][0]),
                              q_contratos_i['tarifa'][0],
                              q_periodos_i_2016,
                              q_periodos_i_2017,
                              q_periodos_i_2018,
                              q_facturas_i['suministro_electrico'],
                              q_facturas_i['punto_suministro'],
                              q_facturas_i['energia_adicional'],
                              q_facturas_i['nota_credito_emitida'],
                              '' ]
              }
    
    df_r2_i = pd.DataFrame(Cliente_i)
    total_2 = total_2.append(df_r2_i)
    
    
    
    
total_3 = pd.DataFrame(columns= ['Item', 'Resultado']) 
for i, fila in q_dadores_r3.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)
    #q_contratos_i = pd.read_sql('SELECT Contratos.contrato_firmado, Contratos.numero_medidor, Contratos.tipo_medidor, Contratos.capacidad_empalme,Contratos.consumo_estimado_invierno,Contratos.consumo_estimado_verano,Contratos.propiedad_medidor,Contratos.tarifa,Contratos.observacion, Contratos.dador_id FROM Contratos WHERE Contratos.dador_id='+str(fila['id_dador']), engine)
    q_contratos_i = pd.read_sql('SELECT Medidores.id_medidor, Medidores.numero_medidor, Medidores.tipo_medidor, Medidores.estado_medidor, Medidores.capacidad_empalme,Medidores.consumo_estimado_invierno,Medidores.consumo_estimado_verano,Medidores.propiedad_medidor,Medidores.tarifa, Medidores.dador_id FROM Medidores WHERE Medidores.dador_id='+str(fila['id_dador']), engine)
    q_facturas_i = pd.read_sql('SELECT * FROM Facturas WHERE Facturas.dador_id=' + str(fila['id_dador']) , engine)
    q_periodos_i_2016 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2016', engine)
    q_periodos_i_2017 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2017', engine)
    q_periodos_i_2018 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2018', engine)
  
    Cliente_i = {
                    'Item':['ID_Cliente_Dador',
                            'Posee_Contrato_Firmado',
                            'Observación',
                            'N_Medidor',
                            'Tipo_Medidor',
                            'Capacidad_Empalme',
                            'Consumo_Estimado_Invierno',
                            'Consumo_Estimado_Verano',
                            'Propiedad_Medidor',
                            'Tarifa_Definida',
                            'Periodo_SLectura_SFacturacion_2016',
                            'Periodo_SLectura_SFacturacion_2017',
                            'Periodo_SLectura_SFacturacion_2018',
                            'Suministro_Electrico',
                            'Punto_Suministro',
                            'Interes_Energia_Adicional',
                            'Nota_Credito_Emitida',
                             ''],
                'Resultado': [str(fila['id_dador']),
                               #q_contratos_i['contrato_firmado'][0],
                              'NO',
                               str('MEDIDOR ' + str(q_contratos_i['estado_medidor'][0])),
                              str(q_contratos_i['id_medidor'][0]),
                              q_contratos_i['tipo_medidor'][0],
                              q_contratos_i['capacidad_empalme'][0],
                              q_contratos_i['consumo_estimado_invierno'][0],
                              q_contratos_i['consumo_estimado_verano'][0],
                              str(fila['id_dador']) + ' - ' + str(fila['nombre_receptor']) + str(q_contratos_i['propiedad_medidor'][0]),
                              q_contratos_i['tarifa'][0],
                              q_periodos_i_2016,
                              q_periodos_i_2017,
                              q_periodos_i_2018,
                              q_facturas_i['suministro_electrico'],
                              q_facturas_i['punto_suministro'],
                              q_facturas_i['energia_adicional'],
                              q_facturas_i['nota_credito_emitida'],
                              '' ]
              }
    
    df_r3_i = pd.DataFrame(Cliente_i)
    total_3 = total_3.append(df_r3_i)
    
    
    

    
    
total_4 = pd.DataFrame(columns= ['Item', 'Resultado']) 
for i, fila in q_dadores_r4.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)
    #q_contratos_i = pd.read_sql('SELECT Contratos.contrato_firmado, Contratos.numero_medidor, Contratos.tipo_medidor, Contratos.capacidad_empalme,Contratos.consumo_estimado_invierno,Contratos.consumo_estimado_verano,Contratos.propiedad_medidor,Contratos.tarifa,Contratos.observacion, Contratos.dador_id FROM Contratos WHERE Contratos.dador_id='+str(fila['id_dador']), engine)
    q_contratos_i = pd.read_sql('SELECT Medidores.id_medidor, Medidores.numero_medidor, Medidores.tipo_medidor, Medidores.estado_medidor, Medidores.capacidad_empalme,Medidores.consumo_estimado_invierno,Medidores.consumo_estimado_verano,Medidores.propiedad_medidor,Medidores.tarifa, Medidores.dador_id FROM Medidores WHERE Medidores.dador_id='+str(fila['id_dador']), engine)
    q_facturas_i = pd.read_sql('SELECT * FROM Facturas WHERE Facturas.dador_id=' + str(fila['id_dador']) , engine)
    q_periodos_i_2016 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2016', engine)
    q_periodos_i_2017 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2017', engine)
    q_periodos_i_2018 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2018', engine)
  
    Cliente_i = {
                    'Item':['ID_Cliente_Dador',
                            'Posee_Contrato_Firmado',
                            'observación',
                            'N_Medidor',
                            'Tipo_Medidor',
                            'Capacidad_Empalme',
                            'Consumo_Estimado_Invierno',
                            'Consumo_Estimado_Verano',
                            'Propiedad_Medidor',
                            'Tarifa_Definida',
                            'Periodo_SLectura_SFacturacion_2016',
                            'Periodo_SLectura_SFacturacion_2017',
                            'Periodo_SLectura_SFacturacion_2018',
                            'Suministro_Electrico',
                            'Punto_Suministro',
                            'Interes_Energia_Adicional',
                            'Nota_Credito_Emitida',
                             ''],
                'Resultado': [str(fila['id_dador']),
                               #q_contratos_i['contrato_firmado'][0],
                              'NO',
                               str('MEDIDOR ' + str(q_contratos_i['estado_medidor'][0])),
                              str(q_contratos_i['id_medidor'][0]),
                              q_contratos_i['tipo_medidor'][0],
                              q_contratos_i['capacidad_empalme'][0],
                              q_contratos_i['consumo_estimado_invierno'][0],
                              q_contratos_i['consumo_estimado_verano'][0],
                              str(fila['id_dador']) + ' - ' + str(fila['nombre_receptor']) + str(q_contratos_i['propiedad_medidor'][0]),
                              q_contratos_i['tarifa'][0],
                              q_periodos_i_2016,
                              q_periodos_i_2017,
                              q_periodos_i_2018,
                              q_facturas_i['suministro_electrico'],
                              q_facturas_i['punto_suministro'],
                              q_facturas_i['energia_adicional'],
                              q_facturas_i['nota_credito_emitida'],
                              '' ]
              }
    
    df_r4_i = pd.DataFrame(Cliente_i)
    total_4 = total_4.append(df_r4_i)    
    
    
    
    
    
total_5 = pd.DataFrame(columns= ['Item', 'Resultado']) 
for i, fila in q_dadores_r5.iterrows():
    q_receptor_i = pd.read_sql('SELECT * FROM Receptores', engine)
    #q_contratos_i = pd.read_sql('SELECT Contratos.contrato_firmado, Contratos.numero_medidor, Contratos.tipo_medidor, Contratos.capacidad_empalme,Contratos.consumo_estimado_invierno,Contratos.consumo_estimado_verano,Contratos.propiedad_medidor,Contratos.tarifa,Contratos.observacion, Contratos.dador_id FROM Contratos WHERE Contratos.dador_id='+str(fila['id_dador']), engine)
    q_contratos_i = pd.read_sql('SELECT Medidores.id_medidor, Medidores.numero_medidor, Medidores.tipo_medidor, Medidores.estado_medidor, Medidores.capacidad_empalme,Medidores.consumo_estimado_invierno,Medidores.consumo_estimado_verano,Medidores.propiedad_medidor,Medidores.tarifa, Medidores.dador_id FROM Medidores WHERE Medidores.dador_id='+str(fila['id_dador']), engine)
    q_facturas_i = pd.read_sql('SELECT * FROM Facturas WHERE Facturas.dador_id=' + str(fila['id_dador']) , engine)
    q_periodos_i_2016 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2016', engine)
    q_periodos_i_2017 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2017', engine)
    q_periodos_i_2018 = pd.read_sql('SELECT CONVERT(varchar, Periodos.fecha_periodo, 103), Periodos.estado_lectura FROM Periodos WHERE Periodos.dador_id=' + str(fila['id_dador']) + ' AND Periodos.anio=2018', engine)
  
    Cliente_i = {
                    'Item':['ID_Cliente_Dador',
                            'Posee_Contrato_Firmado',
                            'Observación',
                            'N_Medidor',
                            'Tipo_Medidor',
                            'Capacidad_Empalme',
                            'Consumo_Estimado_Invierno',
                            'Consumo_Estimado_Verano',
                            'Propiedad_Medidor',
                            'Tarifa_Definida',
                            'Periodo_SLectura_SFacturacion_2016',
                            'Periodo_SLectura_SFacturacion_2017',
                            'Periodo_SLectura_SFacturacion_2018',
                            'Suministro_Electrico',
                            'Punto_Suministro',
                            'Interes_Energia_Adicional',
                            'Nota_Credito_Emitida',
                             ''],
                'Resultado': [str(fila['id_dador']),
                                #q_contratos_i['contrato_firmado'][0],
                              'NO',
                               str('MEDIDOR ' + str(q_contratos_i['estado_medidor'][0])),
                              str(q_contratos_i['id_medidor'][0]),
                              q_contratos_i['tipo_medidor'][0],
                              q_contratos_i['capacidad_empalme'][0],
                              q_contratos_i['consumo_estimado_invierno'][0],
                              q_contratos_i['consumo_estimado_verano'][0],
                              str(fila['id_dador']) + ' - ' + str(fila['nombre_receptor']) + str(q_contratos_i['propiedad_medidor'][0]),
                              q_contratos_i['tarifa'][0],
                              q_periodos_i_2016,
                              q_periodos_i_2017,
                              q_periodos_i_2018,
                              q_facturas_i['suministro_electrico'],
                              q_facturas_i['punto_suministro'],
                              q_facturas_i['energia_adicional'],
                              q_facturas_i['nota_credito_emitida'],
                              '' ]
              }
    
    df_r5_i = pd.DataFrame(Cliente_i)
    total_5 = total_5.append(df_r5_i)
    
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\Kris Sandoval\Desktop\ResultadoTabuladoENEL.xlsx', engine='xlsxwriter')

# Añadir dataframes a las Hojas del Excel
total_1.to_excel(writer, sheet_name='Cliente 1594986', index=False, header=True)  # Default position, cell A1.
total_2.to_excel(writer, sheet_name='Cliente 1595042', index=False, header=True)
total_3.to_excel(writer, sheet_name='Cliente 1677038', index=False, header=True)
total_4.to_excel(writer, sheet_name='Cliente 1677039', index=False, header=True)
total_5.to_excel(writer, sheet_name='Cliente 2878763', index=False, header=True)

#Guardar Excel
writer.save()

In [ ]:
#CONSULTA DADORES CON PROBLEMAS DE LESTURA EN UN PERIODO
res = pd.DataFrame(columns= ['DADOR', 'FECHA PERIODO' , 'CLAVE LECTURA']) 
q_dadores_sin_lectura = pd.read_sql("  SELECT receptor_id, id_dador, CONVERT(varchar, Periodos.fecha_periodo, 103), estado_lectura FROM Periodos LEFT JOIN Dadores ON Periodos.dador_id = Dadores.id_dador LEFT JOIN Receptores ON Dadores.receptor_id = Receptores.id_receptor WHERE estado_lectura <> 'N' and estado_lectura <> '0'", engine)
#q_dadores_sin_lectura

df_query = pd.DataFrame(q_dadores_sin_lectura)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\Kris Sandoval\Desktop\DadoresConProblemasDeLectura.xlsx', engine='xlsxwriter')

# Añadir dataframes a las Hojas del Excel
df_query.to_excel(writer, sheet_name='Clientes Dadores', index=False, header=True)

#Guardar Excel
writer.save()